# Colaboratory Setting

## Drive mount

## Tensorflow gpu setting

# Import Module & Import Dataset

Import Module

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import os
import cv2
from sklearn import preprocessing
from pathlib import Path
from PIL import Image
import torch
! pip install split-folders
import splitfolders
!pip install tqdm
from tqdm import tqdm
from tqdm import trange
import time
from sklearn.model_selection import train_test_split

In [ ]:
top_path='/content/drive/MyDrive/DeepLearning/math_symbols'
symbols_list=os.listdir(top_path)

### Import DataSet

In [ ]:
dataset_path='/content/drive/MyDrive/DeepLearning/math_symbols'
example_image=[]
example_label=[]

for symbols_dir in os.listdir(dataset_path):
    if symbols_dir.split()[0] in symbols_list:
        for image in os.listdir(dataset_path+"/"+symbols_dir):
            example_label.append(symbols_dir.split()[0])
            example_image.append(dataset_path+"/"+symbols_dir+"/"+image)

In [ ]:
x=[]
for path in tqdm(example_image):
    img=cv2.imread(path)
    img=cv2.resize(img,(100,100))
    img=np.array(img)
    x.append(img)

100%|██████████| 65248/65248 [02:16<00:00, 479.46it/s]


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,example_label,test_size=0.4,shuffle=False,random_state=1004)

In [ ]:
x_train=np.array(x_train)
x_test=np.array(x_test)

In [ ]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
print(x_train.shape)
print(x_test.shape)

(39148, 100, 100, 3)
(26100, 100, 100, 3)


In [ ]:
x_train /=255
x_test /=255

In [ ]:
label_encoder=preprocessing.LabelEncoder()
y_train_temp=label_encoder.fit_transform(y_train)
y_test_temp=label_encoder.fit_transform(y_test)

In [ ]:
y_train=tensorflow.keras.utils.to_categorical(y_train_temp,82)
y_test=tensorflow.keras.utils.to_categorical(y_test_temp,82)

In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=(100,100,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same',input_shape=(100,100,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(82,activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 25, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 40000)             0

In [ ]:
history=model.fit(
    x_train,
    y_train,
    batch_size=10,
    epochs=100,
    validation_split=0.25,
    shuffle=True
)

Epoch 1/100


In [ ]:
plt.plot(history.history['accuracy'],label='train',color='red')
plt.plot(history.history['val_accuracy'],label='validation',color='blue')
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='train', color="red")
plt.plot(history.history['val_loss'], label='validation', color="blue")
plt.title('Model loss')
plt.legend(loc='upper left')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
score,acc=model.evaluate(x_test,y_test)
print(score)
print(acc)

In [ ]:
score,acc=model.evaluate(x_train,y_train)
print(score)
print(acc)

In [ ]:
pred=model.predict(x_test)
pred

In [ ]:
image=x_test[1]
plt.imshow(image)

In [ ]:
x_test=np.array(x_test)

In [ ]:
import matplotlib.image as mpimg

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=[24, 12])

count=0

for i in range(1,len(x_test)/1000):
    image=x_test(i)
    pred=model.predict(image)
    result=np.argsort(pred)
    final_label=label_encoder.inverse_transform(result)
    axs[i].plt.imshow(image)
    axs[i].set_title(str('prediction : '+final_label[i]))

